In [4]:
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame({'user_id': [12315133,42645131], 'sum_usd': [120, 280], 'date': ['2025-01-01', '2025-01-05']})
df

,user_id,sum_usd,date
0,12315133,120,2025-01-01
1,42645131,280,2025-01-05
